## Load data

In [ ]:
import pandas as pd

# Load the Excel file and use the first row as headers
file_path = 'data/routes_with_distance.xlsx'  # Update the path as needed
routes = pd.read_excel(file_path, header=0)  # 'header=0' makes the first row the column headers

# Display the header (first few rows)
routes.head()

## Define base

In [ ]:
import openrouteservice

# Initialize the OpenRouteService client with your API key
API_KEY = 'your_api_key'
client = openrouteservice.Client(key=API_KEY)

# Define the fixed address
base_address = "xxx, yyyy, Switzerland"  # Update the address as needed

geocode_fixed = client.pelias_search(base_address)
base_coords = geocode_fixed['features'][0]['geometry']['coordinates']
base_lat, base_lon = base_coords[1], base_coords[0]

## Create local OpenRouteClient
This only work if the openroute service is running on localhost

In [ ]:
import openrouteservice
# Initialize the OpenRouteService client with your API key
client = openrouteservice.Client(base_url='http://localhost:8080/ors')

# Calculate routes
def get_route(lat1, lon1, lat2, lon2):
    return client.directions(
        coordinates=[[lon1, lat1], [lon2, lat2]],
        profile='driving-car',
        format='geojson'
    )

prev_row = None
routes_from_base = {}
for index, row in routes.iterrows():
    if prev_row is None:
        route = get_route(base_lat,base_lon,row['lat'],row['long'])
    else:
        if row['VST'] == 'Schafisheim':
            #Adding the routes back leads to too much
            #route = get_route(prev_row['lat'],prev_row['long'],row['lat'],row['long'])
            pass
        if prev_row['Tournummer'] != row['Tournummer']:
            route = get_route(base_lat,base_lon,row['lat'],row['long'])
    
    if row['Tournummer'] in routes_from_base:
        routes_from_base[row['Tournummer']].append(route)
    else:
        routes_from_base[row['Tournummer']] = [route]
    prev_row = row

In [ ]:
import json
with open('data/routes_from_base.json', 'w') as json_file:
    json.dump(routes_from_base, json_file)

## Create a map of routes from and to the base 

In [ ]:
routes_to_draw = []
for route_name, route_geo_jsons in routes_from_base.items():
    for geo_route in route_geo_jsons:
        coords = [[coord[1], coord[0]] for coord in geo_route['features'][0]['geometry']['coordinates']]
        routes_to_draw.append(coords)
        
routes_to_draw[0]

In [ ]:
import folium
# Create a folium map centered around the fixed address
spider_map = folium.Map(location=[base_lat, base_lon], zoom_start=10)
for route_to_draw in routes_to_draw:
    # Add route to map
    folium.PolyLine(
        locations=[[coord[0], coord[1]] for coord in route_to_draw],
        color='blue',
        weight=5,
        opacity=0.7
    ).add_to(spider_map)
spider_map.save('export/spider_map.html')
spider_map

## Color the segments according to the how many times they are driven
The visualization is clearer in log scale because the segments close to the base are driven so many times

In [ ]:
from collections import Counter
import math

tuples = [tuple(coord) for lst in routes_from_base.values() for sublist in lst for coord in sublist['features'][0]['geometry']['coordinates']]

segment_count = dict(Counter(tuples))
segment_count_log = {key: math.log(value) for key, value in segment_count.items()}

max_occurrence = max(segment_count.values())
max_occurrence_log = max(segment_count_log.values())

In [ ]:
import branca
weighted_spider_map = folium.Map(location=[base_lat, base_lon], zoom_start=10)
for route_to_draw in routes_to_draw:
    # Add route to map
    folium.ColorLine(
        positions=[[coord[0], coord[1]] for coord in route_to_draw],
        colormap = branca.colormap.linear.viridis.scale(0,max_occurrence_log),
        weight=5,
        colors=[segment_count_log[(coord[1],coord[0])] for coord in route_to_draw]
    ).add_to(weighted_spider_map)
weighted_spider_map.save('export/weighted_spider_map.html')
weighted_spider_map